# Development for NeonPandas

In [1]:
import neo4j
import pandas as pd 
from lib.graph import Graph

from utils import df_tools
from utils import cypher
from utils import neo_utils

## Load Pets Dataset

In [2]:
pets = pd.read_csv('pets.csv')
pets

,name,species,color,age,behavior
0,Ralph,Dog,black,10.0,NaN
1,Pip,Cat,yellow,6.0,good
2,Babe,Pig,NaN,3.0,NaN
3,Bubbles,Fish,red,NaN,acceptable
4,Freckles,Horse,brown,NaN,NaN


## Prepare Dataset for NeonPandas

In [3]:
pets = df_tools.set_labels(pets, column='species', labels='Pet')
pets

,labels,name,color,age,behavior
0,"{Pet, Dog}",Ralph,black,10.0,NaN
1,"{Pet, Cat}",Pip,yellow,6.0,good
2,"{Pig, Pet}",Babe,NaN,3.0,NaN
3,"{Pet, Fish}",Bubbles,red,NaN,acceptable
4,"{Pet, Horse}",Freckles,brown,NaN,NaN


## Set up Graph

In [4]:
graph = Graph(uri='bolt://localhost:7687', auth=('neo4j', 'neonpandas'))

### Creates Nodes via DataFrame

In [ ]:
graph.create_nodes(pets)


### Create Constraints via DataFrame

In [ ]:
constraints = pd.DataFrame([
    {'labels': 'Pet', 'property': 'name'}, 
    {'labels': 'Owner', 'property': 'name'}
])
constraints = df_tools.set_labels(constraints, column='labels')
constraints

In [ ]:
# create from dataframe
graph.create_node_constraints(constraints)

### Create Edges

In [ ]:
edges = pd.DataFrame([
    {'src': 'Ralph', 'rel_type': 'SHARES_OWNER', 'dest': 'Bubbles'},
    {'src': 'Pip', 'rel_type': 'SHARES_OWNER', 'dest': 'Babe'},
    {'src': 'Ralph', 'rel_type': 'SHARES_OWNER', 'dest': 'Freckles'}
])
edges

In [ ]:
query = """UNWIND $edges AS edge
        MATCH (s:Pet {name: edge.src})
        MATCH (d:Pet {name: edge.dest})
        WITH s,d,edge
        CALL apoc.merge.relationship(s, edge.rel_type, {}, {}, d) YIELD rel
        RETURN COUNT(rel)"""

In [ ]:
graph.create_relationships(edges, query)

### Node Joining
Perform _join_ operations with an input DataFrame against nodes in Neo4j.

In [ ]:
new_pets = pd.DataFrame([
    {'name': 'Betsy', 'age': 2, 'species': 'Cow'},
    {'name': 'Carrie', 'species': 'Rabbit'}
])
new_pets = df_tools.set_labels(new_pets, labels={'Pet'}, column='species')
all_pets = pd.concat([pets, new_pets], sort=False).reset_index(drop=True)
all_pets

### Semi-Join
Check which nodes in DataFrame exist in Neo4j.

In [ ]:
graph.semi_join(all_pets, on='name', labels='Pet')

### Anti-Join
Check which nodes in DataFrame do not exist in Neo4j.

In [ ]:
graph.anti_join(all_pets, on='name', labels='Pet')

## Match Nodes
Search for nodes via `MATCH` statement

In [5]:
graph.match_nodes(labels={'Pet'}, limit=3)

,labels,color,name,age,behavior
0,"{Pet, Dog}",black,Ralph,10.0,NaN
1,"{Pet, Cat}",yellow,Pip,6.0,good
2,"{Pig, Pet}",NaN,Babe,3.0,NaN


In [7]:
## add properties to search
graph.match_nodes(labels={'Pet'}, properties={'name': 'Ralph'})

,labels,color,name,age
0,"{Pet, Dog}",black,Ralph,10.0
